<a href="https://colab.research.google.com/github/Y4S1-Deep-Learning/Leaf-Desease-Detection---Supervised-Learning/blob/aathif/DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries and mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# List files in the specified directory
!ls "/content/drive/My Drive/DLProject"
!ls "/content/drive/My Drive/DLProject/Dataset"

# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.image import imread
import cv2
import random
import os
from os import listdir
import tensorflow as tf
from keras.preprocessing import image
from tensorflow.keras.utils import img_to_array, array_to_img
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Flatten, Dropout, Dense
from sklearn.model_selection import train_test_split
from keras.models import model_from_json
from keras.utils import to_categorical

# Display the TensorFlow version
print(tf.__version__)

# Plot random images from the dataset
plt.figure(figsize=(12, 12))
path = "/content/drive/My Drive/DLProject/Dataset"
for i in range(1, 17):
    plt.subplot(4, 4, i)
    plt.tight_layout()
    rand_img = imread(path + '/' + random.choice(sorted(os.listdir(path)))
    plt.imshow(rand_img)
    plt.xlabel(rand_img.shape[1], fontsize=10)
    plt.ylabel(rand_img.shape[0], fontsize=10)

# Define a function to convert an image to an array
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None:
            image = cv2.resize(image, (256, 256))
            return img_to_array(image)
        else:
            return np.array([])
    except Exception as e:
        print(f"Error: {e}")
        return None

# Define directory path
dir = "/content/drive/My Drive/DLProject"

# Initialize lists to store image data and labels
image_list, label_list = [], []

# Define all possible labels and their corresponding binary labels
all_labels = ['Apple Scab Leaf', 'Apple leaf', 'Apple rust leaf', ...]  # List of labels
binary_labels = [0, 1, 2, 3, 4, 5, ...]  # Corresponding binary labels

# Initialize a variable to keep track of labels
temp = -1

# Loop through the directories and images
for directory in ['Dataset']:
    plant_image_list = listdir(f"{dir}/{directory}")
    temp += 1
    for files in plant_image_list:
        image_path = f"{dir}/{directory}/{files}"
        image_list.append(convert_image_to_array(image_path))
        label_list.append(binary_labels[temp])

# Create a DataFrame to count the occurrences of each label
label_counts = pd.DataFrame(label_list).value_counts()
label_counts.head()

# Check the shape of the first image in the list
image_list[0].shape

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(image_list, label_list, test_size=0.2, random_state=10)

# Normalize the image data and reshape
x_train = np.array(x_train, dtype=np.float16) / 255.0
x_test = np.array(x_test, dtype=np.float16) / 255.0
x_train = x_train.reshape(-1, 256, 256, 3)
x_test = x_test.reshape(-1, 256, 256, 3)

# Convert labels to categorical format
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Create a Sequential model
model = Sequential()
model.add(Conv2D(32, (3, 3), padding="same", input_shape=(256, 256, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Conv2D(16, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2))
model.add(Flatten())
model.add(Dense(8, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

# Display the model summary
model.summary()

# Compile the model
model.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy'])

# Split the training data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=10)

# Define training parameters
epochs = 25
batch_size = 128

# Train the model
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val))

# Plot the model accuracy over epochs
plt.figure(figsize=(12, 5))
plt.plot(history.history['accuracy'], color='r')
plt.plot(history.history['val_accuracy'], color='b')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'val'])
plt.show()

# Calculate the model's accuracy on the test data
print("Calculating model accuracy")
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1] * 100}")

# Make predictions on the test data
y_pred = model.predict(x_test)

# Display an image from the test data
img = array_to_img(x_test[11])
img

# Display the original and predicted labels for a specific test data point
print("Original Label:", all_labels[np.argmax(y_test[11])])
print("Predicted Label:", all_labels[np.argmax(y_pred[11])])
print(y_pred[2])

# Display the original and predicted labels for a range of test data points
for i in range(50):
    print(all_labels[np.argmax(y_test[i])], "-", all_labels[np.argmax(y_pred[i])])

# Save the model to Google Drive
model.save("/content/drive/My Drive/DLProject/Model")
